# Set run directory

In [189]:
from pathlib import Path
runDir = Path('2018-04-25_17-00-29')
runDir.exists()

True

# Update records

In [190]:
import pandas as pd
from pathlib import Path
from json import load
from os import stat

# Check for errors
if stat(runDir / 'err.txt').st_size != 0:
    raise ValueError('The error file is not empty! Check err.txt!')
if stat(runDir / 'out.txt').st_size != 0:
    raise ValueError('Something was written to standard output! Check out.txt!')

# Open records
records = pd.DataFrame()
if Path('records.csv').exists():
    records = pd.read_csv('records.csv')

with (runDir / '0.json').open('r') as recF: 
    data = load(recF)
    del data['CV sample times [s]']
    del data['CV samples']
    data['Info'] = 'SIAM' # TODO
    
    records = records.append(data, ignore_index=True)
records.tail()

ValueError: The error file is not empty! Check err.txt!

# Save records

In [185]:
records.to_csv('records.csv', index=False)

# Read iterations record

In [186]:
# Read files
itersData = []
for runP in runDir.glob('*.json'):
    if runP.name == 'setup.json': 
        continue
    with runP.open('r') as recF: 
        data = load(recF)
        dataDf = pd.DataFrame(data['CV samples'], 
                              index=data['CV sample times [s]'],
                              columns =['Rank {}'.format(runP.name[:-5])])
        dataDf.index.name = 'Time [s]'
        itersData.append(dataDf)

itersDf = pd.concat(itersData, axis=1)
itersDf.head()

,Rank 0
Time [s],
0.000000,1.000000
0.000005,1.000000
0.213281,0.749498
0.424499,0.718540
0.696465,0.659289


# Save iterations record

In [187]:
itersDf.to_csv('{}.csv'.format(runDir.name))

# Clean run directory

In [188]:
for f in runDir.glob('*'):
    f.unlink()
runDir.rmdir()